# Using a Calculator Tool with Claude
In this recipe, we'll demonstrate how to provide Claude with a simple calculator tool that it can use to perform arithmetic operations based on user input. We'll define the calculator tool and show how Claude can interact with it to solve mathematical problems.

## Step 1: Set up the environment

First, let's install the required libraries and set up the Anthropic API client.

In [1]:
%pip install anthropic

Note: you may need to restart the kernel to use updated packages.


In [2]:
from anthropic import Anthropic
import os
import getpass
os.environ["ANTHROPIC_API_KEY"] = getpass.getpass()

client = Anthropic()
MODEL_NAME = "claude-3-opus-20240229"

 ········


## Step 2: Define the calculator tool
We'll define a simple calculator tool that can perform basic arithmetic operations. The tool will take a mathematical expression as input and return the result. 

Note that we are calling ```eval``` on the outputted expression. This is bad practice and should not be used generally but we are doing it for the purpose of demonstration.

In [3]:
import re

def calculate(expression):
    # Remove any non-digit or non-operator characters from the expression
    expression = re.sub(r'[^0-9+\-*/().]', '', expression)
    
    try:
        # Evaluate the expression using the built-in eval() function
        result = eval(expression)
        return str(result)
    except (SyntaxError, ZeroDivisionError, NameError, TypeError, OverflowError):
        return "Error: Invalid expression"

tools = [
    {
        "name": "calculator",
        "description": "A simple calculator that performs basic arithmetic operations.",
        "input_schema": {
            "type": "object",
            "properties": {
                "expression": {
                    "type": "string",
                    "description": "The mathematical expression to evaluate (e.g., '2 + 3 * 4')."
                }
            },
            "required": ["expression"]
        }
    }
]

In this example, we define a calculate function that takes a mathematical expression as input, removes any non-digit or non-operator characters using a regular expression, and then evaluates the expression using the built-in eval() function. If the evaluation is successful, the result is returned as a string. If an error occurs during evaluation, an error message is returned.

We then define the calculator tool with an input schema that expects a single expression property of type string.

## Step 3: Interact with Claude
Now, let's see how Claude can interact with the calculator tool to solve mathematical problems.

In [4]:
def process_tool_call(tool_name, tool_input):
    if tool_name == "calculator":
        return calculate(tool_input["expression"])

def chat_with_claude(user_message):
    print(f"\n{'='*50}\nUser Message: {user_message}\n{'='*50}")

    message = client.beta.tools.messages.create(
        model=MODEL_NAME,
        max_tokens=4096,
        messages=[{"role": "user", "content": user_message}],
        tools=tools,
    )

    print(f"\nInitial Response:")
    print(f"Stop Reason: {message.stop_reason}")
    print(f"Content: {message.content}")

    if message.stop_reason == "tool_use":
        tool_use = next(block for block in message.content if block.type == "tool_use")
        tool_name = tool_use.name
        tool_input = tool_use.input

        print(f"\nTool Used: {tool_name}")
        print(f"Tool Input: {tool_input}")

        tool_result = process_tool_call(tool_name, tool_input)

        print(f"Tool Result: {tool_result}")

        response = client.beta.tools.messages.create(
            model=MODEL_NAME,
            max_tokens=4096,
            messages=[
                {"role": "user", "content": user_message},
                {"role": "assistant", "content": message.content},
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "tool_result",
                            "tool_use_id": tool_use.id,
                            "content": tool_result,
                        }
                    ],
                },
            ],
            tools=tools,
        )
    else:
        response = message

    final_response = next(
        (block.text for block in response.content if hasattr(block, "text")),
        None,
    )
    print(response.content)
    print(f"\nFinal Response: {final_response}")

    return final_response

## Step 4: Try it out!

Let's try giving Claude a few example math questions now that it has access to a calculator.

In [5]:
chat_with_claude("What is the result of 1,984,135 * 9,343,116?")
chat_with_claude("Calculate (12851 - 593) * 301 + 76")
chat_with_claude("What is 15910385 divided by 193053?")


User Message: What is the result of 1,984,135 * 9,343,116?

Initial Response:
Stop Reason: tool_use
Content: [TextBlock(text='<thinking>\nThe user\'s request is to calculate the result of multiplying 1,984,135 by 9,343,116.\n\nThe calculator tool is relevant for performing this arithmetic operation. Its required parameter is:\nexpression - The mathematical expression to evaluate. \nThe user has provided the numbers to multiply (1,984,135 and 9,343,116), so we can directly populate the expression parameter with "1984135 * 9343116".\n\nNo other tools are needed. We have all the necessary information to proceed with the calculator tool call.\n</thinking>', type='text'), ToolUseBlock(id='toolu_013pBQCR6ftanEkvWugsHHXT', input={'expression': '1984135 * 9343116'}, name='calculator', type='tool_use')]

Tool Used: calculator
Tool Input: {'expression': '1984135 * 9343116'}
Tool Result: 18538003464660
[TextBlock(text='Therefore, the result of 1,984,135 * 9,343,116 is 18,538,003,464,660.', type=

'So 15910385 divided by 193053 is equal to 82.41459599177428.'